In [1]:
import datasets
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,Seq2SeqTrainer,Seq2SeqTrainingArguments
import pandas as pd
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
path=''

In [3]:
dataset = datasets.load_dataset("csv", data_files={"test":[path+"test.csv"]})

Using custom data configuration default-db3f16788d6993e6
Found cached dataset csv (/home/sbaqri/.cache/huggingface/datasets/csv/default-db3f16788d6993e6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained('long-t5-tglobal-base')
model = AutoModelForSeq2SeqLM.from_pretrained('LongT5/t5_final_model/checkpoint-1230')

In [5]:
max_input_length = 2048
max_target_length = 13

def preprocess_function(examples):
    inputs = [doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        label_strings=[doc for doc in examples["target"]] 
        labels = tokenizer(label_strings, max_length=max_target_length,truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/sbaqri/.cache/huggingface/datasets/csv/default-db3f16788d6993e6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-8708f9d7a41cd734.arrow


In [7]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
args=Seq2SeqTrainingArguments(predict_with_generate=True,output_dir='tmp_trainer',per_device_eval_batch_size=8)

In [9]:
trainer = Seq2SeqTrainer(model,args=args,data_collator=data_collator,tokenizer=tokenizer)

In [10]:
test_dataset=tokenized_datasets['test']

In [11]:
raw_preds,_,metrics=trainer.predict(test_dataset,num_beams=2,max_length=13)

The following columns in the test set don't have a corresponding argument in `LongT5ForConditionalGeneration.forward` and have been ignored: text, target. If text, target are not expected by `LongT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1095
  Batch size = 8
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/sbaqri/.conda/envs/LongT5/lib/python3.8/site-packages/transformers/modeling_utils.py:717: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [12]:
raw_preds

array([[    0,     3,    23, ...,     0,     0,     0],
       [    0,    73,  4057, ...,     0,     0,     0],
       [    0,  1393,   125, ...,     0,     0,     0],
       ...,
       [    0,     3, 22686, ...,     0,     0,     0],
       [    0,  1988,   213, ...,     0,     0,     0],
       [    0,  1393,   125, ...,     0,     0,     0]])

In [13]:
decoded_preds = tokenizer.batch_decode(raw_preds, skip_special_tokens=True)

In [14]:
test_dataset=pd.DataFrame(dataset['test'])

In [15]:
test_dataset['predictions']=decoded_preds

In [16]:
test_dataset['Correct']=np.where(test_dataset.target==test_dataset.predictions,1,0)

In [19]:
test_dataset['Correct'].sum()/len(test_dataset['Correct'])

0.6155251141552511

In [20]:
test_dataset['Correct'][test_dataset.target!='unclassified'].sum()/len(test_dataset['Correct'][test_dataset.target!='unclassified'])

0.6458752515090543

In [21]:
acc_per_class=test_dataset.groupby(['predictions']).mean()['Correct']

/tmp/ipykernel_16326/2158407206.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  acc_per_class=test_dataset.groupby(['predictions']).mean()['Correct']


In [22]:
acc_per_class.columns=['Predictions','Accuracy']

In [23]:
acc_per_class=pd.DataFrame(acc_per_class)